<a href="https://colab.research.google.com/github/sarthak-314/100-Days-in-Data/blob/master/2.%20Data%20Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA CLEANING**
***

## Cleaning **Time Sheets**


In [ ]:
#Config
import os 
import pandas as pd
TIME_SHEETS_PATH = './100D Data/TimeSheets'

%cd /content/drive/My\ Drive

/content/drive/My Drive


Loading the files along with their dates in csv file

In [ ]:
os.listdir(TIME_SHEETS_PATH)[:5]

['1st September.csv',
 '29th August.csv',
 '28th August (HD).csv',
 '27th August.csv',
 '26th August.csv']

The dates are not sorted. I need a way to tell the dataframe that 28th August comes before 01st September.

So I will sort the dates according to day of year.

Ex. 1st Jan is 1st day of year and 31st December is 365th day of the year (2020) 

In [ ]:
MONTH_NUMBER = {'August' : 8, 'September' : 9, 'Octuber' : 10, 'October':10 ,'November' : 11, 'December' :  12}
DAYS_IN_PREV_MONTH = {'August' : 31, 'September' : 31, 'Octuber' : 30, 'October':30 , 'November' : 31, 'December' : 30}
def get_day_of_year(file_name):
    file_name = file_name.split('.')[0]
    if 'HD' in file_name: 
        file_name = file_name[:-5]
    date, month = file_name.split(' ')
    if date[:2].isdigit(): 
        date = int(date[:2])
    else: 
        date = int(date[0])
    month_number = MONTH_NUMBER[month]
    days_in_prev_month = DAYS_IN_PREV_MONTH[month]
    return date + (month_number-1) * days_in_prev_month

In [ ]:
time_sheet_csv_files = os.listdir(TIME_SHEETS_PATH)
#Sorting the Time Sheets accroding to day of the year
time_sheet_csv_files.sort(key = lambda x : get_day_of_year(x))
time_sheet_csv_files

['23rd August.csv',
 '24th August.csv',
 '25th August.csv',
 '26th August.csv',
 '27th August.csv',
 '28th August (HD).csv',
 '29th August.csv',
 '30th August (HD).csv',
 '31st August.csv',
 '1st September.csv',
 '2nd September.csv',
 '3rd September.csv',
 '5th September.csv',
 '6th September.csv',
 '7th September.csv',
 '8th September.csv',
 '9th September.csv',
 '10th September.csv',
 '11th September.csv',
 '12th September.csv',
 '13th September.csv',
 '15th September.csv',
 '16th September.csv',
 '17th September.csv',
 '18th September.csv',
 '19th September.csv',
 '22nd September.csv',
 '23rd September.csv',
 '1st October.csv',
 '24th September.csv',
 '2nd October.csv',
 '25th September.csv',
 '3rd October.csv',
 '26th September.csv',
 '4th October.csv',
 '27th September.csv',
 '5th October.csv',
 '28th September.csv',
 '29th September.csv',
 '7th October.csv',
 '8th October.csv',
 '9th October.csv',
 '10th October.csv',
 '11th October.csv',
 '12th October.csv',
 '14th October.csv',

In [ ]:
#Get all the dataframes
df_list = [pd.read_csv(os.path.join(TIME_SHEETS_PATH, x)) for x in time_sheet_csv_files]
date_list = []
half_days = []
for x in time_sheet_csv_files: 
    x = x.split('.')[0]
    if 'HD' in x: 
        x = x[:-5]
        half_days.append(x)
    date_list.append(x)

For each date, I also get the day_number of the 100 day experiment. 

I will store this date to day_number mapping in a DataFrame because I'll need it in next notebook


In [ ]:
df_date = pd.DataFrame()
df_date['day_number'] = ''
for date in date_list: 
    day_number = get_day_of_year(date) - get_day_of_year('23th August') + 1
    df_date.at[date, 'day_number'] = day_number

df_date.to_csv('./100D Data/Cleaned Dataframes/df_date')
df_date.head(6)

,day_number
23rd August,1
24th August,2
25th August,3
26th August,4
27th August,5
28th August,6


**df_date** maps each date to it's day_number (day number of the 100 Day experiment)


For each day, I have a seprate dataframe. I will now stack all the dataframes together to make use of fast pandas vectorized operations

**df** is all the dataframes stacked vertically. This is the *mega dataframe* that contains all the info from all the WorkSheets

In [ ]:
for df, date in zip(df_list, date_list):
    df['date'] = date

df = pd.concat(df_list, axis=0)
df.head()

,time_span,category,task,productivity,date
0,5:15 AM - 6:15 AM,Code,AWS : DO @ I (Learn),8.5,23rd August
1,32 min,Cycling,9.21 km (121 bpm),-,23rd August
2,49 min,Walking,4.31 km (109 bpm),-,23rd August
3,8:00 AM - 10:15 AM,Code,100D : DS @ L,7.5,23rd August
4,10:30 AM,Breakfast,Breakfast,-,23rd August


df now has 4 columns - time_span, category, task, efficiency and date

#### Cleaning **Category**
Each entry has an associated category in the dataframe.

The category tells me wheather I'm coding, exercising, reading etc.

Code for cleaning the category column by making everything consistant -



In [ ]:
EXERCISES = ['cycling', 'workout', 'walking', 'exercise']
def get_category(entry):
    entry = str(entry).strip()
    if 'CT' in entry or 'code' in entry.lower(): 
        return 'Code'
    if 'BF' in entry.upper() or 'breakfast' in entry.lower(): 
        return 'Breakfast'
    if 'lunch' in entry.lower(): 
        return 'Lunch'
    for exercise in EXERCISES: 
        if exercise in entry.lower():
            return exercise.title()
    if 'ML' in entry.upper() or 'meta' in entry.lower(): 
        return 'Metalearning'
    if 'RT' in entry.upper() or 'read' in entry.lower(): 
        return 'Reading'

In [ ]:
df.category = df.category.apply(get_category)
df.head()

,time_span,category,task,productivity,date
0,5:15 AM - 6:15 AM,Code,AWS : DO @ I (Learn),8.5,23rd August
1,32 min,Cycling,9.21 km (121 bpm),-,23rd August
2,49 min,Walking,4.31 km (109 bpm),-,23rd August
3,8:00 AM - 10:15 AM,Code,100D : DS @ L,7.5,23rd August
4,10:30 AM,Breakfast,Breakfast,-,23rd August


### Seprating **df** into *df_code*, *df_exercise* and *df_extra* 

Because coding and exercising are very different categories, I will give them both their own dataframe. (*df_code* and *df_exercise*)

Activities such as reading, eating that are neither coding nor exercising will go into a 3rd dataframe - *df_extra*

### **df_code**
All the time I spent coding goes in here

If you look at **df** above, it has a column for category. I seprate out all the entries that has the category 'Code' into *df_code*

*df_code* now contains all the code related stuff


In [ ]:
df_code = df[df.category == 'Code'].reset_index(drop=True)
df_code = df_code.drop('category', axis=1)
df_code

,time_span,task,productivity,date
0,5:15 AM - 6:15 AM,AWS : DO @ I (Learn),8.5,23rd August
1,8:00 AM - 10:15 AM,100D : DS @ L,7.5,23rd August
2,11:00 AM - 12:00 PM,Leetcode @ I,8,23rd August
3,12:30 PM - 1:30 PM,Book Store : BE @ L (Refactoring),7,23rd August
4,3:00 PM - 5:00 PM,100D : DS @ L (Write),7.5,23rd August
...,...,...,...,...
434,1:00 PM - 2:00 PM,ADE : CLW @ S,6.5,29th November
435,4:30 AM - 5:30 AM,FOLT : CLW @ L,6.5,30th November
436,7:00 AM - 7:45 AM,FOLT : CLW @ L,6.5,30th November
437,10:45 AM - 11:45 AM,OOP : CLW @ S,6,30th November



df_code has a cryptic task column. I'm going to seprate task into 4 columns - project, stack, position, context

 I used the task column to store what I'm working on with :, @, () as seprators. Time to seprate

 Ex. In **"Amatron : ML @ I (Refactoring)"** the column values are - 
 * **working_on** - *(Amatron)* This is what I'm working on 
 * **stack** - *(ML for Machine Learning)* part of the stack I'm working on 
 * **position** - *(I for standing)* am I sitting or standing or laying in bed? 
 * **context** *(optional)* - *[Refactoring]* additional label for context 

In [ ]:
STACK = {'ML': 'Machine Learning', 'DS': 'Data Science', 'BE' : 'Back End', 'FE' : 'Front End', 'BS' : 'Brainstorm', 'R&D' : 'R&D', 'DO':'Dev-Ops', 'A' : 'Algorithms & DS', 'M' : 'Math', 'CLW' : 'College Work'}

In [ ]:
def get_working_on(entry): 
    entry = str(entry)
    entry = entry[ : entry.find('@')]
    entry = entry[ : entry.find(':')]
    working_on = entry.strip().title()
    return working_on

def get_stack(entry):
    entry = str(entry)
    if 'leetcode' in entry.lower(): 
        return STACK['A']
    for stack in list(STACK.keys()): 
        if stack in entry: 
            return STACK[stack]
    return None

#I for standing and L for sitting
#TODO: Change this
def get_position(entry): 
    entry = str(entry)
    if '@' in entry: 
        #.find() returns -1 if string does not exist
        if '(' in entry:
            position = entry[entry.find('@') + 1: entry.find('(')]
        else: 
            position = entry[entry.find('@') + 1 : ]
        position = position.strip()
        if position == 'L': 
            return 'Sitting'
        elif position == 'I':
            return 'Standing'
        elif position == 'S':
            return 'Laying down'
    return None

def get_context(entry):
    entry = str(entry)
    if '(' in entry:
        # from 'Docker @ S (Learn)' => context = 'learn'
        context = entry[entry.find('(') + 1 : entry.find(')')].lower()
        if 'document' in context: 
            return 'Documented'
        if 'refactor' in context: 
            return 'Refactoring'
        return context.title()
    return None


In [ ]:
df_code['working_on'] = df_code.task.apply(get_working_on)
df_code['stack'] = df_code.task.apply(get_stack)
df_code['position'] = df_code.task.apply(get_position)
df_code['context'] = df_code.task.apply(get_context)
df_code = df_code.drop('task', axis=1)
#Reordering columns
cols = df_code.columns.to_list()
cols = [cols[0]] + cols[3:] + cols[1:3]
df_code = df_code[cols]

df_code.head()


,time_span,working_on,stack,position,context,productivity,date
0,5:15 AM - 6:15 AM,Aws,Dev-Ops,Standing,Learn,8.5,23rd August
1,8:00 AM - 10:15 AM,100D,Data Science,Sitting,None,7.5,23rd August
2,11:00 AM - 12:00 PM,Leetcode,Algorithms & DS,Standing,None,8,23rd August
3,12:30 PM - 1:30 PM,Book Store,Back End,Sitting,Refactoring,7,23rd August
4,3:00 PM - 5:00 PM,100D,Data Science,Sitting,Write,7.5,23rd August


### **df_exercise**
(Future Sarthak : Ignore the rest of the notebook. I have automated the collection process for smartwatch)

Seprating out the exercise category from df

In [ ]:
def is_exercise(entry):
    for exercise in EXERCISES: 
        if exercise in str(entry).lower(): 
            return True
    return False
df_exercise = df[df.category.apply(is_exercise)]
df_exercise = df_exercise.rename(columns={'category' : 'activity', 'task': 'measurement'})
df_exercise = df_exercise.drop('productivity', axis=1)
def get_measurement(x):
    try: 
        if '(' in x: 
            return get_context(x).split(' ')[0]
    except: 
        return ''
df_exercise['bpm'] = df_exercise.measurement.apply(get_measurement)
df_exercise.measurement = df_exercise.measurement.apply(lambda x : x[ : x.find('(')])
#Reordering columns
cols = df_exercise.columns.to_list()
cols = cols[:-2] + [cols[-1]] + [cols[-2]]
df_exercise = df_exercise[cols]
df_exercise = df_exercise.reset_index(drop=True)
df_exercise.head()

*df_exercise* contains 5 columns - time_span, activity, measurement, bpm (heart beats per minute) and date. The columns are self-descriptive

###**df_extra**
*df_extra* for those entries who couldn't make it into either df_code or df_exercise

This contains categories like reading, eating etc.

In [ ]:
df_not_code = df[df.category != 'Code']
#check if not exercise
df_not_exercise = df_not_code.category.apply(lambda x : not is_exercise(x))
df_extra = df_not_code[df_not_exercise]
df_extra = df_extra.drop('productivity', axis = 1)
df_extra.head()

df_extra contins stuff like breakfast time, lunch time, reading time etc

Save df_code, df_exercise and df_extra into csv files

In [ ]:
SAVE_PATH = './100D Data/Cleaned Dataframes'
df_code.to_csv(os.path.join(SAVE_PATH, 'df_code'), index = False)
df_exercise.to_csv(os.path.join(SAVE_PATH, 'df_exercise'), index=False)
df_extra.to_csv(os.path.join(SAVE_PATH, 'df_extra'), index=False)

To recap, first I combined all the worksheets by stacking their dataframes vertically.

Then I seprated code, exercise and other category from the mega dataframe and saved them into my Google Drive

## Cleaning **Smartwatch Sheets**

In [ ]:
SMARTWATCH_SHEETS_PATH = './100D Data/Smartwatch Sheets'

In [ ]:
os.listdir(SMARTWATCH_SHEETS_PATH)

Sorting files on basis of #week

In [ ]:
def get_week_number(entry):
    file_name = entry.split('.')[0].strip()
    week = file_name[-2:].strip()
    return int(week)

In [ ]:
csv_file_names = os.listdir(SMARTWATCH_SHEETS_PATH)
csv_file_names.sort(key = lambda x : get_week_number(x))
df_list = [pd.read_csv(os.path.join(SMARTWATCH_SHEETS_PATH, x)) for x in csv_file_names]
week_list = [x.split('.')[0].split(' ')[1] for x in csv_file_names]
df_list[0]

Stacking all the dataframes for all the weeks vertically

In [ ]:
for df, week in zip(df_list, week_list):
    df['week'] = week

df_smartwatch = pd.concat(df_list, axis=0)
df_smartwatch

That's it for the Smartwatch Sheets. Let's save it

In [ ]:
SAVE_PATH = './100D Data/Cleaned Dataframes'
df_smartwatch.to_csv(os.path.join(SAVE_PATH, 'df_smartwatch'), index=False)